In [ ]:
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
import joblib
from keras import backend as K
from tensorflow.keras.utils import plot_model
K.set_image_data_format('channels_first')

In [2]:
VQA_model_file_name      = 'models/VQA/VQA_MODEL.json'
VQA_weights_file_name   = 'models/VQA/VQA_MODEL_WEIGHTS.hdf5'
label_encoder_file_name  = 'models/VQA/FULL_labelencoder_trainval.pkl'
CNN_weights_file_name   = 'models/CNN/vgg16_weights.h5'

In [3]:
def get_image_model(CNN_weights_file_name):
    from models.CNN.VGG import VGG_16
    image_model = VGG_16(CNN_weights_file_name)
    image_model.layers.pop()
    image_model.layers.pop()
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    image_model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return image_model

In [ ]:
model_vgg = get_image_model(CNN_weights_file_name)

In [5]:
def get_image_features(image_file_name):
    image_features = np.zeros((1, 4096))
    im = cv2.resize(cv2.imread(image_file_name), (224, 224))
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0) 
    image_features[0,:] = model_vgg.predict(im)[0]
    return image_features

In [6]:
def get_question_features(question):
    word_embeddings = spacy.load('en_vectors_web_lg')
    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, 30, 300))
    for j in range(len(tokens)):
        question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

In [19]:
def get_VQA_model(VQA_model_file_name, VQA_weights_file_name):
    vqa_model = model_from_json(open(VQA_model_file_name).read())
    vqa_model.load_weights(VQA_weights_file_name)
    vqa_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return vqa_model

In [ ]:
model_vqa = get_VQA_model(VQA_model_file_name, VQA_weights_file_name)

In [14]:
image_file_name = 'test.jpg'
question = u"What vehicle is in the picture?"

<img src="test.jpg">

In [15]:
image_features = get_image_features(image_file_name)

In [16]:
question_features = get_question_features(question)

In [5]:
y_output = model_vqa.predict([question_features, image_features])

labelencoder = joblib.load(label_encoder_file_name)
for label in reversed(np.argsort(y_output)[0,-5:]):
    print(str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label))

51.87 %  train
031.5 %  bicycle
03.81 %  bike
02.91 %  bus
02.54 %  scooter
